# 데이터 확인

In [1]:
import os
import json
import pandas as pd

# 최상위 경로 설정
root_dir = '../data/CAT_image'
json_count = 0


# 하위 폴더 개수 세기
folder_count = sum(
    os.path.isdir(os.path.join(root_dir, name))
    for name in os.listdir(root_dir)
)

print(f"📁 폴더 개수: {folder_count}개")

for folder_name in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder_name)

    if os.path.isdir(folder_path):
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.json'):
                json_count += 1

print(f"📦 JSON 파일 개수: {json_count}개")

📁 폴더 개수: 664개
📦 JSON 파일 개수: 664개


## Step01 : 전체 데이터 확인 및 라벨 전처리

In [75]:
# 데이터 상태 확인용도로 함수 정의함
def print_label_distribution(df):
    label_columns = ['action', 'emotion', 'situation']
    for col in label_columns:
        print(f"🔹 {col} 분포:")
        print(df[col].value_counts())
        print()
        
        unique_labels = sorted(df[col].unique())
        print(f"🔹 {col} 라벨 셋 ({len(unique_labels)}개):")
        for label in unique_labels:
            print(f"  - {label}")
        print("\n")

import re

def preprocess_situation_label(label: str) -> str:
    # 감정 접두사 제거 (예: "공격성_", "공포_", "불안/슬픔_", "편안/안정_", "행복/즐거움_", "화남/불쾌_")
    label = re.sub(r'^(공격성|공포|불안/슬픔|편안/안정|행복/즐거움|화남/불쾌)_', '', label)
    label = label.strip()
    
    # 산책 관련 라벨 통일
    if label in [
        "산책이나 노즈워크 중",
        "산책 준비 또는 산책중일 때",
        "산책 준비",
        "산책 중/산책 준비",
        "산책 나왔을 때"
    ]:
        label = "산책 중/산책 준비"
    
    # 필요 시 추가 전처리 규칙 삽입 가능
    
    return label

def preprocess_label(label: str) -> str:
    import re
    # 1) 감정 접두사 제거
    label = re.sub(r'^(공격성|공포|불안/슬픔|편안/안정|행복/즐거움|화남/불쾌)_', '', label)
    label = label.strip()
    return label

In [40]:
# 정규화할 라벨 매핑 정의 : 태윤님 라벨링 작업과 Align
action_label_map = {
    "걷거나 뜀": "걷거나 달리는 동작",
    "걷거나 뜀 ": "걷거나 달리는 동작",
    "걷거나 뛰는 동작": "걷거나 달리는 동작",

    "꼬리를 흔든다": "꼬리를 흔드는 동작",
    "꼬리를 흔드는 동작": "꼬리를 흔드는 동작",

    "납작 엎드림": "납작 엎드리는 동작",
    "납작 엎드리는 동작": "납작 엎드리는 동작",

    "배를 보임": "배를 보여주는 동작",
    "배를 보이는 동작": "배를 보여주는 동작",
    "배를 보여주는 동작": "배를 보여주는 동작",

    "그루밍함": "그루밍하는 동작",
    "그루밍하는 동작": "그루밍하는 동작",

    "머리를 들이댐": "머리를 들이대는 동작",
    "머리를 들이대는 동작": "머리를 들이대는 동작",

    "발을 숨기고 웅크리고 앉음": "발을 숨기고 웅크리고 앉는 동작",
    "발을 숨기고 웅크리고 앉는 동작": "발을 숨기고 웅크리고 앉는 동작",

    "앞발로 꾹꾹 누름": "앞발로 꾹꾹 누르는 동작",
    "앞발로 꾹꾹 누르는 동작": "앞발로 꾹꾹 누르는 동작",

    "옆으로 누워 있음": "옆으로 눕는 동작",
    "옆으로 눕는 동작": "옆으로 눕는 동작",

    "좌우로 뒹굴음": "좌우로 뒹구는 동작",
    "좌우로 뒹구는 동작": "좌우로 뒹구는 동작",

    "팔을 뻗어 휘적거림": "팔을 뻗어 휘적거리는 동작",
    "앞발을 뻗어 휘적거리는 동작": "팔을 뻗어 휘적거리는 동작",

    "허리를 아치로 세움": "허리를 아치로 세우는 동작",
    "허리를 아치로 세우는 동작": "허리를 아치로 세우는 동작"
}

In [41]:
situation_label_map = {
    # 기타
    "공격성_기타": "기타",
    "공포_기타": "기타",
    "불안/슬픔_기타": "기타",
    "편안/안정_기타": "기타",
    "행복/즐거움_기타": "기타",
    "화남/불쾌_기타": "기타",

    # 낯선 소리 관련
    "공격성_초인종 소리가 났을 때": "낯선 소리",
    "공격성_낯선 소리가 났을 때": "낯선 소리",
    "공격성_낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 소리",
    "공포_초인종 소리가 났을 때": "낯선 소리",
    "공포_낯선 소리가 났을 때": "낯선 소리",
    "공포_낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 소리",
    "불안/슬픔_낯선 소리가 났을 때": "낯선 소리",
    "불안/슬픔_낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 소리",
    "편안/안정_낯선 소리가 났을 때": "낯선 소리",
    "편안/안정_낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 소리",
    "행복/즐거움_초인종 소리가 났을 때": "낯선 소리",
    "행복/즐거움_낯선 소리가 났을 때": "낯선 소리",
    "행복/즐거움_낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 소리",
    "화남/불쾌_초인종 소리가 났을 때": "낯선 소리",
    "화남/불쾌_낯선 소리가 났을 때": "낯선 소리",
    "화남/불쾌_낯선 소리가 나거나 낯선 사람을 봤을 때": "낯선 소리",

    # 낯선 사람/동물/장소 관련
    "공격성_낯선 도구를 자신의 몸에 사용할 때(미용도구 등)": "미용/위생관리",
    "공격성_빗질/발톱깍기/목욕 등 위생관리를 할 때": "미용/위생관리",
    "공포_낯선 도구를 자신의 몸에 사용할 때(미용도구)": "미용/위생관리",
    "공포_빗질/발톱깍기/목욕 등 위생관리를 할 때": "미용/위생관리",
    "불안/슬픔_빗질/발톱깍기/목욕 등 위생관리를 할 때": "미용/위생관리",
    "편안/안정_낯선 도구를 자신의 몸에 사용할 때(미용도구 등)": "미용/위생관리",
    "편안/안정_빗질/발톱깍기/목욕 등 위생관리를 할 때": "미용/위생관리",
    "행복/즐거움_빗질/발톱깍기/목욕 등 위생관리를 할 때": "미용/위생관리",
    "화남/불쾌_빗질/발톱깍기/목욕 등 위생관리를 할 때": "미용/위생관리",
    "목욕할 때": "미용/위생관리",
    "목욕하거나 싫어하는 부위를 만질 때" : "미용/위생관리",
    
    "공격성_낯선 동물 또는 사람을 만났을 때": "낯선 사람/동물",
    "공격성_다른 동물을 보거나 낯선 사람을 만날 때": "낯선 사람/동물",
    "공격성_다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "낯선 사람/동물",
    "공포_다른 사람이나 동물을 만났을 때": "낯선 사람/동물",
    "공포_낯선 동물 또는 사람을 만났을 때": "낯선 사람/동물",
    "불안/슬픔_다른 동물을 보거나 낯선 사람을 만날 때": "낯선 사람/동물",
    "불안/슬픔_다른 사람이나 동물을 만났을 때": "낯선 사람/동물",
    "편안/안정_다른 동물을 보거나 낯선 사람을 만날 때": "낯선 사람/동물",
    "편안/안정_다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "낯선 사람/동물",
    "편안/안정_다른 사람이나 동물을 만났을 때": "낯선 사람/동물",
    "행복/즐거움_다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "낯선 사람/동물",
    "행복/즐거움_다른 사람이나 동물을 만났을 때": "낯선 사람/동물",
    "화남/불쾌_낯선 동물 또는 사람을 만났을 때": "낯선 사람/동물",
    "화남/불쾌_다른 동물을 보거나 낯선 사람을 만날 때": "낯선 사람/동물",
    "화남/불쾌_다른 동물을 보거나 낯선 사람을 만날 때 산책 나왔을 때": "낯선 사람/동물",
    "화남/불쾌_다른 사람이나 동물을 만났을 때": "낯선 사람/동물",

    "공격성_낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소",
    "공포_낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소",
    "불안/슬픔_낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소",
    "편안/안정_낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소",
    "행복/즐거움_낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소",
    "화남/불쾌_낯선 장소에 있거나 낯선 소리가 날 때": "낯선 장소",

    # 산책 관련
    "공격성_산책이나 노즈워크 중": "산책 중/산책 준비",
    "불안/슬픔_산책 준비 또는 산책중일 때": "산책 중/산책 준비",
    "불안/슬픔_산책이나 노즈워크 중": "산책 중/산책 준비",
    "편안/안정_산책 준비 또는 산책중일 때": "산책 중/산책 준비",
    "편안/안정_산책이나 노즈워크 중": "산책 중/산책 준비",
    "행복/즐거움_산책 준비 또는 산책중일 때": "산책 중/산책 준비",
    "행복/즐거움_산책이나 노즈워크 중": "산책 중/산책 준비",
    "화남/불쾌_산책 준비 또는 산책중일 때": "산책 중/산책 준비",
    "화남/불쾌_산책이나 노즈워크 중": "산책 중/산책 준비",

    # 먹을 것/장난감 관련 (추가)
    "공격성_먹을것, 장난감이 앞에 있을 때": "먹을 것/장난감이 앞에 있을 때",
    "공포_먹을것, 장난감이 앞에 있을 때": "먹을 것/장난감이 앞에 있을 때",
    "불안/슬픔_먹을것, 장난감이 앞에 있을 때": "먹을 것/장난감이 앞에 있을 때",
    "편안/안정_먹을것, 장난감이 앞에 있을 때": "먹을 것/장난감이 앞에 있을 때",
    "행복/즐거움_먹을것, 장난감이 앞에 있을 때": "먹을 것/장난감이 앞에 있을 때",
    "화남/불쾌_먹을것, 장난감이 앞에 있을 때": "먹을 것/장난감이 앞에 있을 때",

    # 싫어하는 부위 접촉 (추가)
    "공격성_싫어하는 부위를 만질 때": "싫어하는 부위 접촉",
    "불안/슬픔_싫어하는 부위를 만질 때": "싫어하는 부위 접촉",
    "편안/안정_싫어하는 부위를 만질 때": "싫어하는 부위 접촉",
    "행복/즐거움_싫어하는 부위를 만질 때": "싫어하는 부위 접촉",
    "화남/불쾌_싫어하는 부위를 만질 때": "싫어하는 부위 접촉",

    # 보호자 관련
    "공격성_보호자가 집에 돌아왔을 때": "보호자 돌아옴",
    "공포_보호자가 집에 돌아왔을 때": "보호자 돌아옴",
    "불안/슬픔_보호자가 집에 돌아왔을 때": "보호자 돌아옴",
    "편안/안정_보호자가 집에 돌아왔을 때": "보호자 돌아옴",
    "행복/즐거움_보호자가 집에 돌아왔을 때": "보호자 돌아옴",
    "화남/불쾌_보호자가 집에 돌아왔을 때": "보호자 돌아옴",

    "공격성_보호자에게 혼났을 때": "보호자에게 혼남",
    "불안/슬픔_보호자에게 혼났을 때": "보호자에게 혼남",
    "편안/안정_보호자에게 혼났을 때": "보호자에게 혼남",
    "화남/불쾌_보호자에게 혼났을 때": "보호자에게 혼남",
    "혼날 때" : "보호자에게 혼남",

    "불안/슬픔_보호자와 떨어지거나 혼자 남겨졌을 때": "보호자와 분리",
    "편안/안정_보호자와 떨어지거나 혼자 남겨졌을 때": "보호자와 분리",
    "행복/즐거움_보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때": "보호자와 분리",
    "화남/불쾌_보호자와 떨어질 때/혼자 남겨지거나 낯선장소에 있을 때": "보호자와 분리",

    # 편안한 접촉
    "공격성_편안히 쓰다듬어 줄 때": "편안한 접촉",
    "불안/슬픔_편안히 쓰다듬어 줄 때": "편안한 접촉",
    "편안/안정_편안히 쓰다듬어 줄 때": "편안한 접촉",
    "행복/즐거움_편안히 쓰다듬어 줄 때": "편안한 접촉",
    "화남/불쾌_편안히 쓰다듬어 줄 때": "편안한 접촉",

    # 잠들기 전/같이 누움
    "공격성_잠들기 전이나 같이 누워있을 때": "잠들기 전/같이 누움",
    "불안/슬픔_잠들기 전이나 같이 누워있을 때": "잠들기 전/같이 누움",
    "편안/안정_잠들기 전이나 같이 누워있을 때": "잠들기 전/같이 누움",
    "행복/즐거움_잠들기 전이나 같이 누워있을 때": "잠들기 전/같이 누움",
    "화남/불쾌_잠들기 전이나 같이 누워있을 때": "잠들기 전/같이 누움",

    # 휴식/자기 공간
    "공격성_휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)": "휴식/자기 공간",
    "불안/슬픔_휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)": "휴식/자기 공간",
    "편안/안정_휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)": "휴식/자기 공간",
    "행복/즐거움_휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)": "휴식/자기 공간",
    "화남/불쾌_휴식시간, 자신만의 공간에 들어갔을 때(캔넬, 소파 침대 밑 등)": "휴식/자기 공간",
}


In [42]:
# CSV 파일 읽기
# df = pd.read_csv("../data/meta_info.csv")
# for col in ['action', 'emotion', 'situation']:
#    df[col] = df[col].astype(str).str.strip()
# df.head()

In [43]:
# df['action'] = df['action'].apply(lambda x: action_label_map.get(x, x))
# df['situation'] = df['situation'].apply(lambda x: situation_label_map.get(x, x))

# print_label_distribution(df)

In [44]:
# df.to_csv("remeta_info.csv", index=False)

In [45]:
df = pd.read_csv("../data/remeta_info.csv")
print_label_distribution(df)

🔹 action 분포:
action
그루밍하는 동작             5184
꼬리를 흔드는 동작           3106
팔을 뻗어 휘적거리는 동작       2948
걷거나 달리는 동작           2576
옆으로 눕는 동작            1734
발을 숨기고 웅크리고 앉는 동작    1470
납작 엎드리는 동작           1105
배를 보여주는 동작            940
머리를 들이대는 동작           865
앞발로 꾹꾹 누르는 동작         770
좌우로 뒹구는 동작            666
허리를 아치로 세우는 동작        180
Name: count, dtype: int64

🔹 action 라벨 셋 (12개):
  - 걷거나 달리는 동작
  - 그루밍하는 동작
  - 꼬리를 흔드는 동작
  - 납작 엎드리는 동작
  - 머리를 들이대는 동작
  - 발을 숨기고 웅크리고 앉는 동작
  - 배를 보여주는 동작
  - 앞발로 꾹꾹 누르는 동작
  - 옆으로 눕는 동작
  - 좌우로 뒹구는 동작
  - 팔을 뻗어 휘적거리는 동작
  - 허리를 아치로 세우는 동작


🔹 emotion 분포:
emotion
편안/안정     15363
행복/즐거움     3907
공격성        1273
화남/불쾌       666
불안/슬픔       278
공포           57
Name: count, dtype: int64

🔹 emotion 라벨 셋 (6개):
  - 공격성
  - 공포
  - 불안/슬픔
  - 편안/안정
  - 행복/즐거움
  - 화남/불쾌


🔹 situation 분포:
situation
먹을 것/장난감                  5310
기타                        4851
휴식/자기 공간                  4524
잠들기 전/같이 누움               2801
편안한 접촉                    1317
보호자 돌아옴          

### Step02 : 프레임 수 고려해서 train/val/test dataset Setting

In [46]:
df['frames'].describe()

count    21544.000000
mean        99.179075
std         20.598154
min          1.000000
25%         84.000000
50%         95.000000
75%        110.000000
max        185.000000
Name: frames, dtype: float64

In [47]:
df_filtered = df[(df['frames'] >= 80) & (df['frames'] <= 150)]
df_filtered['frames'].describe()

count    18417.000000
mean       101.744041
std         16.999153
min         80.000000
25%         88.000000
50%         98.000000
75%        112.000000
max        150.000000
Name: frames, dtype: float64

In [48]:
print_label_distribution(df_filtered)

🔹 action 분포:
action
그루밍하는 동작             4622
꼬리를 흔드는 동작           2556
팔을 뻗어 휘적거리는 동작       2514
걷거나 달리는 동작           1979
옆으로 눕는 동작            1526
발을 숨기고 웅크리고 앉는 동작    1321
납작 엎드리는 동작            957
배를 보여주는 동작            808
머리를 들이대는 동작           739
앞발로 꾹꾹 누르는 동작         671
좌우로 뒹구는 동작            570
허리를 아치로 세우는 동작        154
Name: count, dtype: int64

🔹 action 라벨 셋 (12개):
  - 걷거나 달리는 동작
  - 그루밍하는 동작
  - 꼬리를 흔드는 동작
  - 납작 엎드리는 동작
  - 머리를 들이대는 동작
  - 발을 숨기고 웅크리고 앉는 동작
  - 배를 보여주는 동작
  - 앞발로 꾹꾹 누르는 동작
  - 옆으로 눕는 동작
  - 좌우로 뒹구는 동작
  - 팔을 뻗어 휘적거리는 동작
  - 허리를 아치로 세우는 동작


🔹 emotion 분포:
emotion
편안/안정     13305
행복/즐거움     3222
공격성        1065
화남/불쾌       551
불안/슬픔       239
공포           35
Name: count, dtype: int64

🔹 emotion 라벨 셋 (6개):
  - 공격성
  - 공포
  - 불안/슬픔
  - 편안/안정
  - 행복/즐거움
  - 화남/불쾌


🔹 situation 분포:
situation
먹을 것/장난감                  4429
기타                        4196
휴식/자기 공간                  3993
잠들기 전/같이 누움               2486
편안한 접촉                    1082
보호자 돌아옴          

In [49]:
# df_filtered.to_csv("df_filtered.csv", index=False)

situation과 emotion조합으로 train/val로 나눌려고 했으나 샘플수가 적어서 pass

In [50]:
# stratify 기준 컬럼 생성
df_filtered['stratify_col'] = df_filtered['situation'] + "_" + df_filtered['emotion']

combo_counts =df_filtered['stratify_col'].value_counts()
combo_counts

/tmp/ipykernel_1487702/2888848573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['stratify_col'] = df_filtered['situation'] + "_" + df_filtered['emotion']


stratify_col
휴식/자기 공간_편안/안정       3905
기타_편안/안정             3532
잠들기 전/같이 누움_편안/안정    2463
먹을 것/장난감_행복/즐거움      2350
먹을 것/장난감_편안/안정       1100
                     ... 
낯선 장소_공포                2
낯선 사람/동물_공포             1
보호자 돌아옴_공포              1
보호자와 분리_화남/불쾌           1
보호자에게 혼남_행복/즐거움         1
Name: count, Length: 80, dtype: int64

In [51]:
"""
낯선 사람/동물_공포             1
보호자 돌아옴_공포              1
보호자와 분리_화남/불쾌           1
보호자에게 혼남_행복/즐거움         1

제거 필요
"""

'\n낯선 사람/동물_공포             1\n보호자 돌아옴_공포              1\n보호자와 분리_화남/불쾌           1\n보호자에게 혼남_행복/즐거움         1\n\n제거 필요\n'

In [52]:
valid_combos = combo_counts[combo_counts >= 2].index
df_valid = df_filtered[df_filtered['stratify_col'].isin(valid_combos)]

In [53]:
print_label_distribution(df_valid)
df_valid

🔹 action 분포:
action
그루밍하는 동작             4622
꼬리를 흔드는 동작           2555
팔을 뻗어 휘적거리는 동작       2514
걷거나 달리는 동작           1978
옆으로 눕는 동작            1526
발을 숨기고 웅크리고 앉는 동작    1321
납작 엎드리는 동작            957
배를 보여주는 동작            808
머리를 들이대는 동작           739
앞발로 꾹꾹 누르는 동작         671
좌우로 뒹구는 동작            569
허리를 아치로 세우는 동작        153
Name: count, dtype: int64

🔹 action 라벨 셋 (12개):
  - 걷거나 달리는 동작
  - 그루밍하는 동작
  - 꼬리를 흔드는 동작
  - 납작 엎드리는 동작
  - 머리를 들이대는 동작
  - 발을 숨기고 웅크리고 앉는 동작
  - 배를 보여주는 동작
  - 앞발로 꾹꾹 누르는 동작
  - 옆으로 눕는 동작
  - 좌우로 뒹구는 동작
  - 팔을 뻗어 휘적거리는 동작
  - 허리를 아치로 세우는 동작


🔹 emotion 분포:
emotion
편안/안정     13305
행복/즐거움     3221
공격성        1065
화남/불쾌       550
불안/슬픔       239
공포           33
Name: count, dtype: int64

🔹 emotion 라벨 셋 (6개):
  - 공격성
  - 공포
  - 불안/슬픔
  - 편안/안정
  - 행복/즐거움
  - 화남/불쾌


🔹 situation 분포:
situation
먹을 것/장난감                  4429
기타                        4196
휴식/자기 공간                  3993
잠들기 전/같이 누움               2486
편안한 접촉                    1082
보호자 돌아옴          

,video_name,frames,action,emotion,situation,stratify_col
0,20201028_cat-arch-000156.mp4,80,허리를 아치로 세우는 동작,화남/불쾌,낯선 장소,낯선 장소_화남/불쾌
3,20201115_cat-arch-001607.mp4,94,허리를 아치로 세우는 동작,행복/즐거움,먹을 것/장난감,먹을 것/장난감_행복/즐거움
4,20201118_cat-arch-000248.mp4,97,허리를 아치로 세우는 동작,편안/안정,먹을 것/장난감,먹을 것/장난감_편안/안정
6,20201120_cat-arch-001449.mp4,99,허리를 아치로 세우는 동작,행복/즐거움,먹을 것/장난감,먹을 것/장난감_행복/즐거움
7,20201123_cat-arch-000924.mp4,84,허리를 아치로 세우는 동작,공포,낯선 소리,낯선 소리_공포
...,...,...,...,...,...,...
21538,cat-walkrun-100983,91,걷거나 달리는 동작,공격성,먹을 것/장난감,먹을 것/장난감_공격성
21540,cat-walkrun-102261,103,걷거나 달리는 동작,편안/안정,기타,기타_편안/안정
21541,cat-walkrun-102641,136,걷거나 달리는 동작,편안/안정,기타,기타_편안/안정
21542,cat-walkrun-102718,122,걷거나 달리는 동작,편안/안정,기타,기타_편안/안정


In [54]:
# df_valid.to_csv("../data/df_filtered_2nd.csv", index=False)

In [55]:
# stratified split
from sklearn.model_selection import train_test_split
# stratified split
train_val, test = train_test_split(
    df_valid, test_size=0.2, random_state=42,
    stratify=df_valid['stratify_col']
)

train, val = train_test_split(
    train_val,
    test_size=0.25,  # 0.25 * 0.8 = 0.2
    random_state=42,
    stratify=train_val['stratify_col']
)


In [56]:
# 또는 개별 label별 분포도 확인 가능
print("\n[Train] situation:\n", train['stratify_col'].value_counts(normalize=True))
print("[Train] emotion:\n", train['stratify_col'].value_counts(normalize=True))


[Train] situation:
 stratify_col
휴식/자기 공간_편안/안정       0.212094
기타_편안/안정             0.191817
잠들기 전/같이 누움_편안/안정    0.133701
먹을 것/장난감_행복/즐거움      0.127636
먹을 것/장난감_편안/안정       0.059745
                       ...   
보호자에게 혼남_불안/슬픔       0.000181
낯선 장소_공포             0.000181
낯선 장소_행복/즐거움         0.000181
미용/위생관리_행복/즐거움       0.000181
낯선 장소_공격성            0.000181
Name: proportion, Length: 76, dtype: float64
[Train] emotion:
 stratify_col
휴식/자기 공간_편안/안정       0.212094
기타_편안/안정             0.191817
잠들기 전/같이 누움_편안/안정    0.133701
먹을 것/장난감_행복/즐거움      0.127636
먹을 것/장난감_편안/안정       0.059745
                       ...   
보호자에게 혼남_불안/슬픔       0.000181
낯선 장소_공포             0.000181
낯선 장소_행복/즐거움         0.000181
미용/위생관리_행복/즐거움       0.000181
낯선 장소_공격성            0.000181
Name: proportion, Length: 76, dtype: float64


In [57]:
train = train.drop(columns=['stratify_col'])
val = val.drop(columns=['stratify_col'])
test = test.drop(columns=['stratify_col'])

In [58]:
# train, val, test
# train.to_csv("train_fin.csv", index=False)
# val.to_csv("val_fin.csv", index=False)
# test.to_csv("test_fin.csv", index=False)

### step03 : 샘플링 전랴

In [59]:
# stratify 기준 컬럼 생성
df_valid['stratify_col'] = df_valid['situation'] + "_" + df_valid['emotion']

combo_counts =df_valid['stratify_col'].value_counts()
# combo_counts

/tmp/ipykernel_1487702/616519498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['stratify_col'] = df_valid['situation'] + "_" + df_valid['emotion']


In [60]:
import pandas as pd

def show(col,df):
    # 1) 각 클래스별 샘플 수 확인
    class_counts = df[col].value_counts()
    print(class_counts)

    # 2) 언더샘플링 기준: 소수 클래스 샘플 수로 맞추기 (최소 샘플 수)
    min_count = class_counts.min()
    print(f"언더샘플링 기준 샘플 수: {min_count}\n")

    # 3) 클래스별로 샘플을 랜덤하게 min_count 만큼 추출해서 합치기
    df_under = pd.concat([
        df[df[col] == cls].sample(min_count, random_state=42)
        for cls in class_counts.index
    ])

    # 4) 결과 확인
    print(df_under[col].value_counts())

In [61]:
col = 'stratify_col'
show(col,df_valid)

stratify_col
휴식/자기 공간_편안/안정       3905
기타_편안/안정             3532
잠들기 전/같이 누움_편안/안정    2463
먹을 것/장난감_행복/즐거움      2350
먹을 것/장난감_편안/안정       1100
                     ... 
산책 중/산책 준비_공격성          2
낯선 장소_공격성               2
낯선 장소_공포                2
미용/위생관리_공포              2
낯선 소리_행복/즐거움            2
Name: count, Length: 76, dtype: int64
언더샘플링 기준 샘플 수: 2

stratify_col
낯선 소리_행복/즐거움         2
휴식/자기 공간_편안/안정       2
기타_편안/안정             2
잠들기 전/같이 누움_편안/안정    2
보호자 돌아옴_불안/슬픔        2
                    ..
보호자 돌아옴_편안/안정        2
산책 중/산책 준비_편안/안정     2
기타_행복/즐거움            2
보호자 돌아옴_행복/즐거움       2
싫어하는 부위 접촉_화남/불쾌     2
Name: count, Length: 76, dtype: int64


In [62]:
import pandas as pd

def under_sampling_with_threshold(df, stratify_col, threshold=10, multiplier=3, random_state=42):
    class_counts = df[stratify_col].value_counts()
    
    # 기준 이상인 클래스만 선택
    valid_classes = class_counts[class_counts >= threshold].index.tolist()
    print(f"기준 이상 클래스 수: {len(valid_classes)} / 전체 클래스 수: {len(class_counts)}")
    
    target_count = threshold * multiplier
    print(f"샘플링 목표 수: {target_count}")
    
    sampled_df_list = []
    for cls in valid_classes:
        cls_df = df[df[stratify_col] == cls]
        n_samples = min(len(cls_df), target_count)
        sampled = cls_df.sample(n_samples, random_state=random_state)
        sampled_df_list.append(sampled)
    
    # 기준 미만 클래스는 원본 그대로 포함할지 선택 (여기서는 포함)
    small_class_df = df[~df[stratify_col].isin(valid_classes)]
    sampled_df_list.append(small_class_df)
    
    result_df = pd.concat(sampled_df_list).reset_index(drop=True)
    
    print("언더샘플링 후 클래스별 샘플 수:")
    print(result_df[stratify_col].value_counts())
    
    return result_df



In [63]:
# 사용 예
new_dataset = under_sampling_with_threshold(df_valid, 'stratify_col', threshold=10, multiplier=3)


기준 이상 클래스 수: 48 / 전체 클래스 수: 76
샘플링 목표 수: 30
언더샘플링 후 클래스별 샘플 수:
stratify_col
먹을 것/장난감_불안/슬픔    30
휴식/자기 공간_화남/불쾌    30
편안한 접촉_행복/즐거움     30
낯선 장소_화남/불쾌       30
보호자 돌아옴_행복/즐거움    30
                  ..
보호자와 분리_행복/즐거움     2
낯선 장소_공포           2
미용/위생관리_공포         2
낯선 장소_공격성          2
산책 중/산책 준비_공격성     2
Name: count, Length: 76, dtype: int64


In [64]:
print_label_distribution(new_dataset)

🔹 action 분포:
action
꼬리를 흔드는 동작           394
걷거나 달리는 동작           187
팔을 뻗어 휘적거리는 동작       187
그루밍하는 동작             111
납작 엎드리는 동작           106
옆으로 눕는 동작             67
좌우로 뒹구는 동작            60
발을 숨기고 웅크리고 앉는 동작     57
머리를 들이대는 동작           56
배를 보여주는 동작            34
허리를 아치로 세우는 동작        33
앞발로 꾹꾹 누르는 동작          8
Name: count, dtype: int64

🔹 action 라벨 셋 (12개):
  - 걷거나 달리는 동작
  - 그루밍하는 동작
  - 꼬리를 흔드는 동작
  - 납작 엎드리는 동작
  - 머리를 들이대는 동작
  - 발을 숨기고 웅크리고 앉는 동작
  - 배를 보여주는 동작
  - 앞발로 꾹꾹 누르는 동작
  - 옆으로 눕는 동작
  - 좌우로 뒹구는 동작
  - 팔을 뻗어 휘적거리는 동작
  - 허리를 아치로 세우는 동작


🔹 emotion 분포:
emotion
편안/안정     408
화남/불쾌     276
행복/즐거움    238
불안/슬픔     179
공격성       166
공포         33
Name: count, dtype: int64

🔹 emotion 라벨 셋 (6개):
  - 공격성
  - 공포
  - 불안/슬픔
  - 편안/안정
  - 행복/즐거움
  - 화남/불쾌


🔹 situation 분포:
situation
기타                        160
먹을 것/장난감                  155
휴식/자기 공간                  115
밥그릇, 장난감과 같은 소유물을 만질 때     97
낯선 사람/동물                   97
낯선 소리                      90
미용/위생관리         

In [65]:
# stratified split
from sklearn.model_selection import train_test_split
# stratified split
train_val, test = train_test_split(
    new_dataset, test_size=0.2, random_state=42,
    stratify=new_dataset['stratify_col']
)

train, val = train_test_split(
    train_val,
    test_size=0.25,  # 0.25 * 0.8 = 0.2
    random_state=42,
    stratify=train_val['stratify_col']
)


In [66]:
# 또는 개별 label별 분포도 확인 가능
print("\n[Train] situation:\n", train['stratify_col'].value_counts(normalize=True))
print("[Train] emotion:\n", train['stratify_col'].value_counts(normalize=True))


[Train] situation:
 stratify_col
먹을 것/장난감_행복/즐거움               0.023077
기타_공격성                        0.023077
먹을 것/장난감_화남/불쾌                0.023077
먹을 것/장난감_편안/안정                0.023077
밥그릇, 장난감과 같은 소유물을 만질 때_공격성    0.023077
                                ...   
싫어하는 부위 접촉_불안/슬픔              0.001282
산책 중/산책 준비_공격성                0.001282
미용/위생관리_행복/즐거움                0.001282
낯선 소리_행복/즐거움                  0.001282
잠들기 전/같이 누움_공격성               0.001282
Name: proportion, Length: 76, dtype: float64
[Train] emotion:
 stratify_col
먹을 것/장난감_행복/즐거움               0.023077
기타_공격성                        0.023077
먹을 것/장난감_화남/불쾌                0.023077
먹을 것/장난감_편안/안정                0.023077
밥그릇, 장난감과 같은 소유물을 만질 때_공격성    0.023077
                                ...   
싫어하는 부위 접촉_불안/슬픔              0.001282
산책 중/산책 준비_공격성                0.001282
미용/위생관리_행복/즐거움                0.001282
낯선 소리_행복/즐거움                  0.001282
잠들기 전/같이 누움_공격성               0.001282
Name: proportion, Length: 76, dt

In [67]:
train = train.drop(columns=['stratify_col'])
val = val.drop(columns=['stratify_col'])
test = test.drop(columns=['stratify_col'])

In [68]:
# train, val, test
# train.to_csv("train_under.csv", index=False)
# val.to_csv("val_under.csv", index=False)
# test.to_csv("test_under.csv", index=False)

---

In [81]:

df = pd.read_csv("../data/df_filtered_2nd.csv")
unique_names_list = df['video_name'].unique()
df

,video_name,frames,action,emotion,situation,stratify_col
0,20201028_cat-arch-000156.mp4,80,허리를 아치로 세우는 동작,화남/불쾌,낯선 장소,낯선 장소_화남/불쾌
1,20201115_cat-arch-001607.mp4,94,허리를 아치로 세우는 동작,행복/즐거움,먹을 것/장난감,먹을 것/장난감_행복/즐거움
2,20201118_cat-arch-000248.mp4,97,허리를 아치로 세우는 동작,편안/안정,먹을 것/장난감,먹을 것/장난감_편안/안정
3,20201120_cat-arch-001449.mp4,99,허리를 아치로 세우는 동작,행복/즐거움,먹을 것/장난감,먹을 것/장난감_행복/즐거움
4,20201123_cat-arch-000924.mp4,84,허리를 아치로 세우는 동작,공포,낯선 소리,낯선 소리_공포
...,...,...,...,...,...,...
18408,cat-walkrun-100983,91,걷거나 달리는 동작,공격성,먹을 것/장난감,먹을 것/장난감_공격성
18409,cat-walkrun-102261,103,걷거나 달리는 동작,편안/안정,기타,기타_편안/안정
18410,cat-walkrun-102641,136,걷거나 달리는 동작,편안/안정,기타,기타_편안/안정
18411,cat-walkrun-102718,122,걷거나 달리는 동작,편안/안정,기타,기타_편안/안정


In [82]:
# df['video_name'] = df['video_name'].str.strip()

# 유니크 비디오 이름 개수 확인
print("🎞️ 유니크 video_name 개수 (strip 후):", df['video_name'].nunique())

🎞️ 유니크 video_name 개수 (strip 후): 18413


In [77]:
import os

# CAT_image 폴더에서 실제 폴더 이름 리스트 (폴더만 추출)
folder_list = [f for f in os.listdir('../data/CAT_image_2nd') if os.path.isdir(os.path.join('../data/CAT_image_2nd', f))]
folder_set = set(folder_list)

# df에서 추출한 video_name에서 확장자 제거 (예: .mp4 제거)
df_video_names = set(df['video_name'].str.replace('.mp4', '', regex=False))

# 누락된 것 확인
missing_in_df = folder_set - df_video_names
missing_in_folder = df_video_names - folder_set

print(f"📁 폴더에는 있지만 df에는 없는 항목 수: {len(missing_in_df)}")
print(f"📄 df에는 있지만 폴더에는 없는 항목 수: {len(missing_in_folder)}")


📁 폴더에는 있지만 df에는 없는 항목 수: 607
📄 df에는 있지만 폴더에는 없는 항목 수: 4691


In [79]:
import os
import pandas as pd

# df 불러오기
df = pd.read_csv("../data/df_filtered_2nd.csv")

# video_name 전처리 (.mp4 제거, strip)
df_video_names = set(df['video_name'].str.strip().str.replace('.mp4', '', regex=False))

# CAT_image 내 폴더명
base_path = '../data/CAT_image_2nd'
folder_list = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
folder_set = set(folder_list)

# ✅ 폴더는 있지만 df에는 없는 항목
missing_in_df = folder_set - df_video_names

# ✅ df에는 있지만 폴더가 없는 항목
missing_in_folder = df_video_names - folder_set

# ✅ JSON이 존재하지 않는 폴더 목록
missing_json = []
for folder in folder_list:
    json_path = os.path.join(base_path, folder, folder + '.json')
    if not os.path.isfile(json_path):
        missing_json.append(folder)

# 🔍 결과 출력
print(f"📁 폴더 개수: {len(folder_set)}")
print(f"📄 df 내 video_name 개수: {len(df_video_names)}")
print(f"❌ 폴더에는 있지만 df에는 없는 항목 수: {len(missing_in_df)}")
print(f"❌ df에는 있지만 폴더에는 없는 항목 수: {len(missing_in_folder)}")
print(f"❌ JSON 파일이 없는 폴더 수: {len(missing_json)}")

# 필요시 출력
# print("JSON 없는 폴더 목록:", missing_json)


📁 폴더 개수: 14329
📄 df 내 video_name 개수: 18413
❌ 폴더에는 있지만 df에는 없는 항목 수: 607
❌ df에는 있지만 폴더에는 없는 항목 수: 4691
❌ JSON 파일이 없는 폴더 수: 0


In [80]:
# 전체 비디오 이름 수
total = len(df['video_name'])

# 고유한 비디오 이름 수
unique = df['video_name'].nunique()

print(f"전체 video_name 수: {total}")
print(f"유일한 video_name 수: {unique}")

# 중복된 비디오 이름 목록 출력
duplicate_names = df['video_name'][df['video_name'].duplicated()].unique()
print(f"중복된 video_name 수: {len(duplicate_names)}")
print("🔁 중복된 video_name 목록 예시:")
print(duplicate_names[:10])  # 처음 10개만 보기


전체 video_name 수: 18413
유일한 video_name 수: 18413
중복된 video_name 수: 0
🔁 중복된 video_name 목록 예시:
[]


In [83]:
import os

root_path = '../data/CAT_image_2nd'  # 확인하려는 폴더 경로

subfolders = [f for f in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, f))]
print(f"📁 하위 폴더 개수: {len(subfolders)}개")


📁 하위 폴더 개수: 14329개
